In [13]:
from networks_bf import get_all_devices_combined
from services_bf import get_all_services
import itertools
import numpy as np

for c in range(1, 101):
    
    print(f"Processing config-{c}....")
    
    physical_machines = get_all_devices_combined(f"config-{c}")
    services = get_all_services(f"config-{c}")
    
    # Define the problem-specific parameters
    n = len(physical_machines)  # Number of physical machines
    s = len(services)  # Number of services
    
    vm_list = []
    
    for p in range(len(physical_machines)):
        for v in range(len(physical_machines[p].guest_machines)):
            vm_list.append([p, v])
    
    # Calculate all permutations of vm_list with s places (with repetition)
    permutations = itertools.product(vm_list, repeat=s)
    
    # Variable number of virtual machines for each physical machine
    max_v = max(map(lambda machine: len(machine.guest_machines), physical_machines))
    
    ################ Physical and virtual machine configurations ##################
    
    # Initialization
    R = np.empty(shape=(n, max_v))  # Request network delay of the machine
    R.fill(999999) # Initialize with very large value
    S = np.empty(shape=(n, max_v))  # Response network delay of the machine
    S.fill(999999) # Initialize with very large value
    X = np.empty(shape=(n, max_v))  # Maximum IPS possible
    X.fill(1) # Initialize with 1
    
    PI = np.empty(shape=(n, max_v))  # Power consumption in idle
    PI.fill(999) # Initialize with very large value
    PM = np.empty(shape=(n, max_v))  # Maximum power consumption
    PM.fill(999) # Initialize with very large value
    
    # Fill in with the actual numbers
    for i in range(len(physical_machines)):
        for j in range(len(physical_machines[i].guest_machines)):
            R[i][j] = physical_machines[i].guest_machines[j].net_delay_request
            S[i][j] = physical_machines[i].guest_machines[j].net_delay_response
            X[i][j] = physical_machines[i].guest_machines[j].max_instructions_per_second
            PI[i][j] = physical_machines[i].guest_machines[j].idle_cpu_utilization
            PM[i][j] = physical_machines[i].guest_machines[j].max_cpu_utilization
    
    ################ Service configurations ###################
    
    # Acceptable latencies for services
    SAL = [service.acceptable_latency for service in services]
    # Initialize IPS values for services (replace with actual data)
    SIPS = [service.average_instructions_per_second for service in services]
    # Layers considered for service
    SL = [service.layer for service in services]
    
    perm_list = list(permutations)
    
    # Define minimum state variables
    min_latency = float("inf")
    min_energy = float("inf")
    min_objective = float("inf")
    physical_placement = []
    vm_placement = []
    
    for perm in perm_list:
        # a1 = np.array([(0, 0), (2, 1), (1, 0), (0, 1), (0, 0), (1, 1), (0, 0), (3, 1)])
        # a1 = np.array([(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)])
        # a2 = np.array(perm)
        # 
        # if (a1 == a2).all():
        #     print("Found")
        
        I = np.empty(shape=(n, max_v))  # IPS currently executed
        I.fill(0) # Initialize with 0
        # Calculate energy consumption based on the parameters
        E = PI + (PM - PI) * (I / X)
    
        total_latency = 0
        total_energy = 0
        for p in range(len(physical_machines)):
                    for v in range(len(physical_machines[p].guest_machines)):
                        total_energy += E[p][v]
        
        invalid = False # Discard the permutation if it is invalid
        
        for i in range(s):
            p = perm[i][0]
            v = perm[i][1]
            
            vm = physical_machines[p].guest_machines[v]
            
            if SL[i] is not None and SL[i] != vm.layer: # Consider the layer constrains provided by the service
                invalid = True
                break
            
            latency = R[p][v] + (SIPS[i] * 1000 / X[p][v]) + S[p][v]
            if latency > SAL[i]:
                invalid = True
                break
                
            total_latency += latency
            I[p][v] = I[p][v] + SIPS[i] # Update the current instructions per second
            energy = PI[p][v] + (PM[p][v] - PI[p][v]) * (I[p][v] / X[p][v])
            total_energy += energy  # Calculate energy consumption
            
        
        # Something abnormally wrong happened
        if invalid:
            continue
        
        objective = 0.5 * total_latency + 0.5 * total_energy # Only alpha = 0.5 is considered
        if min_objective > objective:
            min_objective = objective
            physical_placement = [elem[0] for elem in perm]
            vm_placement = [elem[1] for elem in perm]
            min_latency = total_latency
            min_energy = total_energy
            
    print(f"Minimum latency: {min_latency}, minimum energy: {min_energy}, physical placement: {physical_placement}, vm placement: {vm_placement}")
        
        

Processing config-1....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-2....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-3....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-4....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-5....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-6....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-7....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-8....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-9....
Minimum latency: inf, minimum energy: inf, physical placement: [], vm placement: []
Processing config-10....
Min